# Neuron Network - Lab

### Part 1: Load  data

Import "bank-data.csv"

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import pandas as pd
bankData = pd.read_csv('/content/gdrive/MyDrive/bank-data.csv', sep = ';')
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


### Part 2: Preprocess data

Preprocess the dataset as you have done before

#### 2.1 Binary encoding

Use LabelEncoder to encode the following columns:
- y
- default
- housing
- loan

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#example
bankData['y'] = le.fit_transform(bankData['y'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [9]:
#Encode the remaining columns
bankData['housing'] = le.fit_transform(bankData['housing'])
bankData['default'] = le.fit_transform(bankData['default'])
bankData['loan'] = le.fit_transform(bankData['loan'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


#### 2.2 Convert categorical variables into dummy columns

(1) Use pd.get_dummies to convert the following categorical variales into dummy columns
- job
- maritial
- education
- contact
- month
- poutcome

(2) Drop columns that have been converted

In [10]:
#example
bankData = pd.concat([bankData,pd.get_dummies(bankData['job'],prefix='job')],axis=1)
bankData = bankData.drop(columns=['job'])
bankData.head()

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,30,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0,0,0,0,0,0,0,0,0,0,0,1,0
1,33,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0,0,0,0,0,0,0,0,1,0,0,0,0
2,35,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0,0,0,0,0,1,0,0,0,0,0,0,0
3,30,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0,0,0,0,0,1,0,0,0,0,0,0,0
4,59,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0,0,1,0,0,0,0,0,0,0,0,0,0


In [11]:
bankData = pd.concat([bankData,pd.get_dummies(bankData['marital'],prefix='marital')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['education'],prefix='education')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['contact'],prefix='contact')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['month'],prefix='month')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['poutcome'],prefix='poutcome')],axis=1)

bankData = bankData.drop(columns=['marital', 'education', 'contact', 'month', 'poutcome'])

In [12]:
bankData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  4521 non-null   int64
 1   default              4521 non-null   int64
 2   balance              4521 non-null   int64
 3   housing              4521 non-null   int64
 4   loan                 4521 non-null   int64
 5   day                  4521 non-null   int64
 6   duration             4521 non-null   int64
 7   campaign             4521 non-null   int64
 8   pdays                4521 non-null   int64
 9   previous             4521 non-null   int64
 10  y                    4521 non-null   int64
 11  job_admin.           4521 non-null   uint8
 12  job_blue-collar      4521 non-null   uint8
 13  job_entrepreneur     4521 non-null   uint8
 14  job_housemaid        4521 non-null   uint8
 15  job_management       4521 non-null   uint8
 16  job_retired          452

#### 2.3 Train/Test separation

Perform hold-out method
- 60% training set
- 40% testing set

In [13]:
bankData_train = bankData.sample(frac = 0.6)
bankData_test = bankData.drop(bankData_train.index)
print(pd.crosstab(bankData_train['y'],columns = 'count'))
print(pd.crosstab(bankData_test['y'],columns = 'count'))

col_0  count
y           
0       2408
1        305
col_0  count
y           
0       1592
1        216


##### X/y separation

In [14]:
bankData_train_y = bankData_train['y']
bankData_train_X = bankData_train.copy()
del bankData_train_X['y']

bankData_test_y = bankData_test['y']
bankData_test_X = bankData_test.copy()
del bankData_test_X['y']

#### 2.4 Feature Scaling

It is always a good practice to scale the features so that all of them can be uniformly evaluated

In [15]:
from sklearn import preprocessing

standard_scaler = preprocessing.StandardScaler()
train_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_train_X), columns=bankData_train_X.columns)
test_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_test_X), columns=bankData_train_X.columns)

min_max_scaler = preprocessing.MinMaxScaler()
train_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_train_X),columns=bankData_train_X.columns)
test_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_test_X),columns=bankData_train_X.columns)

In [16]:
train_X_scaled_s.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,-0.214092,-0.129871,-0.797319,0.873403,-0.424356,1.679050,-0.454513,0.733080,2.056252,0.912491,-0.347957,-0.522600,-0.19256,-0.154187,-0.515162,-0.232382,-0.19461,-0.327638,-0.128396,2.190207,-0.166287,-0.092467,-0.365034,0.779859,-0.593669,-0.418291,0.959890,-0.642667,-0.199655,0.759819,-0.272865,-0.649544,-0.270519,-0.396143,-0.066654,-0.216993,5.337521,-0.437584,-0.372772,-0.103946,-0.662765,-0.310099,-0.131331,-0.098368,2.873915,-0.216993,-0.166287,-2.135281
1,0.735444,-0.129871,-0.560679,0.873403,-0.424356,-1.225166,-0.661035,-0.249875,-0.403706,-0.326232,-0.347957,-0.522600,-0.19256,-0.154187,-0.515162,-0.232382,-0.19461,-0.327638,-0.128396,2.190207,-0.166287,-0.092467,-0.365034,-1.282283,1.684441,-0.418291,0.959890,-0.642667,-0.199655,-1.316102,-0.272865,1.539541,-0.270519,-0.396143,-0.066654,-0.216993,-0.187353,-0.437584,2.682606,-0.103946,-0.662765,-0.310099,-0.131331,-0.098368,-0.347957,-0.216993,-0.166287,0.468322
2,0.165723,-0.129871,-0.416766,0.873403,-0.424356,-0.862139,-0.430684,-0.249875,-0.403706,-0.326232,-0.347957,-0.522600,-0.19256,-0.154187,1.941137,-0.232382,-0.19461,-0.327638,-0.128396,-0.456578,-0.166287,-0.092467,-0.365034,0.779859,-0.593669,-0.418291,-1.041786,1.556016,-0.199655,-1.316102,-0.272865,1.539541,-0.270519,-0.396143,-0.066654,-0.216993,-0.187353,-0.437584,-0.372772,-0.103946,1.508831,-0.310099,-0.131331,-0.098368,-0.347957,-0.216993,-0.166287,0.468322
3,-0.878767,-0.129871,-0.430119,0.873403,2.356510,0.589969,-0.684864,-0.249875,-0.403706,-0.326232,-0.347957,-0.522600,-0.19256,-0.154187,-0.515162,-0.232382,-0.19461,3.052145,-0.128396,-0.456578,-0.166287,-0.092467,-0.365034,0.779859,-0.593669,-0.418291,0.959890,-0.642667,-0.199655,-1.316102,-0.272865,1.539541,-0.270519,-0.396143,-0.066654,-0.216993,-0.187353,-0.437584,-0.372772,-0.103946,1.508831,-0.310099,-0.131331,-0.098368,-0.347957,-0.216993,-0.166287,0.468322
4,-0.119138,7.699928,-0.524701,0.873403,-0.424356,-1.225166,-0.772239,-0.249875,-0.403706,-0.326232,-0.347957,1.913508,-0.19256,-0.154187,-0.515162,-0.232382,-0.19461,-0.327638,-0.128396,-0.456578,-0.166287,-0.092467,-0.365034,0.779859,-0.593669,2.390679,-1.041786,-0.642667,-0.199655,-1.316102,-0.272865,1.539541,-0.270519,-0.396143,-0.066654,-0.216993,-0.187353,-0.437584,-0.372772,-0.103946,1.508831,-0.310099,-0.131331,-0.098368,-0.347957,-0.216993,-0.166287,0.468322


In [17]:
train_X_scaled_m.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.298507,0.0,0.045179,1.0,0.0,0.966667,0.057096,0.081633,0.28555,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.447761,0.0,0.066577,1.0,0.0,0.166667,0.035889,0.020408,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.358209,0.0,0.079591,1.0,0.0,0.266667,0.059543,0.020408,0.00000,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.194030,0.0,0.078383,1.0,1.0,0.666667,0.033442,0.020408,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.313433,1.0,0.069831,1.0,0.0,0.166667,0.024470,0.020408,0.00000,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Artificial Neural Network : sklearn

### Part 3: Train a model

In [18]:
from sklearn.neural_network import MLPClassifier  
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
mlp.fit(train_X_scaled_s, bankData_train_y) 

MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

### Part 4: Model Evaluation

Evaluation metrics
- confusion metrix
- accuracy
- precision, recall, f1-score

### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

## Artificial Neural Network : keras

Fitting a logistic regression model

### Part 3: Train a model

In [19]:
import keras
from keras import models
from keras import layers

In [20]:
train_X_scaled_s.shape

(2713, 48)

In [21]:
nn = models.Sequential()
nn.add(layers.Dense(48,activation = 'linear',input_shape=(None,48)))
nn.add(layers.Dense(1,activation = 'sigmoid'))

In [22]:
nn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 48)          2352      
                                                                 
 dense_1 (Dense)             (None, None, 1)           49        
                                                                 
Total params: 2,401
Trainable params: 2,401
Non-trainable params: 0
_________________________________________________________________


In [27]:
import numpy as np
X_train_add = np.expand_dims(train_X_scaled_s, axis=0)
y_train_add = np.expand_dims(bankData_train_y, axis=0)
y_train_add = np.expand_dims(y_train_add, axis=2)

In [28]:
y_train_add.shape

(1, 2713, 1)

In [29]:
history = nn.fit(X_train_add,y_train_add,epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.8234 - accuracy: 0.5304
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.8170 - accuracy: 0.5345
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 0.8107 - accuracy: 0.5367
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 0.8045 - accuracy: 0.5411
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 0.7984 - accuracy: 0.5440
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.7924 - accuracy: 0.5488
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 0.7864 - accuracy: 0.5503
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.7806 - accuracy: 0.5533
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.7748 - accuracy: 0.5569
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 0.7692 - accuracy: 0.5610
Epoch 11/100
1/1 [===

### Part 4: Model Evaluation

In [32]:
X_test_add = np.expand_dims(test_X_scaled_s, axis=0)
y_test_add = np.expand_dims(bankData_test_y, axis=0)
y_test_add = np.expand_dims(y_test_add, axis=2)

In [33]:
test_loss, test_acc = nn.evaluate(X_test_add, y_test_add)
print('Test Loss: %s\nTest Accuracy: %s' % (test_loss,test_acc))

1/1 [==============================] - 0s 149ms/step - loss: 0.4822 - accuracy: 0.8108
Test Loss: 0.4822392463684082
Test Accuracy: 0.8108407258987427


### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?